# Data analysis of powder XRD data

In [1]:
import glob
import os
import re

import numpy as np
import pandas as pd
import scipy.signal as sps

from matplotlib import pyplot as plt

In [2]:
os.chdir(r'C:\Users\Pierre\Desktop\Postdoc\YTmVO4\YTmVO4_neutrons\2021_ORNL_powder_ND')

## Reference spectra taken from the ICDD

In [3]:
ref_files = glob.glob('**/*_ICSD_*.xy', recursive=True)

In [23]:
dref = {}
for file in ref_files:
    fm = re.match('\d{4}-\d{2}-\d{2}\w+\\\\([^\W_]+)_*', file)
    compound = fm.group(1)
    with open(file, 'r'):
        dref[compound] = pd.read_csv(file, sep='\s+', header=0, names=['2theta', 'I', 'std?'])
        
dref[compound]

,2theta,I,std?
0,1.01,0.10870,0.0
1,1.02,0.10665,0.0
2,1.03,0.10465,0.0
3,1.04,0.10271,0.0
4,1.05,0.10083,0.0
...,...,...,...
11894,119.95,0.22601,0.0
11895,119.96,0.22209,0.0
11896,119.97,0.21937,0.0
11897,119.98,0.21810,0.0


In [5]:
xlabel = r'2$\theta$ (°)'
ylabel = 'I (a.u.)'

In [29]:
# %matplotlib
fig = plt.figure()
for cpd in dref.keys():
    plt.plot(dref[cpd]['2theta'], dref[cpd]['I'], label=cpd)
    plt.xlim([0,75])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title('XRD pattern of TmVO4 and YVO4 as reported in the ICSD database')
    plt.legend()

## Experimental powder XRD data

In [7]:
xp_files = glob.glob('**/*-background.ASC', recursive=True)

In [14]:
compounds = {}
dxp = {}
for file in xp_files:
    fm = re.match('\d{4}-\d{2}-\d{2}\w+\\\\(\d{4}-\d{2}-\d{2})_([^\W_]+)-([^\W_]+)\w+-\w+.ASC', file)
    date = fm.group(1)
    batch = fm.group(3)
    compounds[batch] = fm.group(2)
    with open(file, 'r'):
        dxp[batch,date] = pd.read_csv(file, sep=' ', header=0, names=['2theta', 'I0', 'Estimated Std'])

dxp[batch,date]

,2theta,I0,Estimated Std
0,15.021162,-13.682541,27.718510
1,15.035162,6.051848,28.072201
2,15.049162,7.002535,28.089099
3,15.063162,13.370152,28.202188
4,15.077162,51.255014,28.866011
...,...,...,...
4279,74.927162,11.493725,23.865214
4280,74.941162,-0.987545,23.781568
4281,74.955162,2.412970,24.030383
4282,74.969162,7.257812,24.306377


In [9]:
batches = ['LS5695', 'LS5701', 'LS5696', 'LS5697']

In [15]:
for key in dxp.keys():
    I = dxp[key]['I0']
    Iref = np.mean(I)
    mult = 1.3
    Im = np.mean(I[I<Iref*mult])
    ii = 0
    while abs(Iref-Im)/Iref>0.01:
        Iref = Im
        Im = np.mean(I[I<Iref*mult])
        ii = ii+1
        if ii>100:
            break
        print(ii, Iref, Im)

    dxp[key]['I'] = dxp[key]['I0']-Im
    dxp[key]

1 315.1011523919672 310.24887172674687
2 310.24887172674687 309.9170183031991
1 309.18051098084044 303.62188803157835
2 303.62188803157835 303.0619490611636
1 313.865644417357 304.34229103662386
2 304.34229103662386 303.17062259656973
1 312.3195552760272 307.2008893883647
2 307.2008893883647 306.7936270683599
1 13.816051295977827 1.8230688728870728
2 1.8230688728870728 -6.475294086073686
3 -6.475294086073686 -15.44917951099048
1 15.999686442274745 3.3611809720295986
2 3.3611809720295986 -4.286340254717107
3 -4.286340254717107 -12.381660456214075


## Compare measured and reported XRD patterns for pure TmVO4

In [36]:
# Full spectra
ref = 'TmVO4'
Tm_key = ('LS5695', '2021-03-12')
fig = plt.figure()
plt.plot(dref[ref]['2theta'], dref[ref]['I'], label='ICSD')
plt.plot(dxp[Tm_key]['2theta'], dxp[Tm_key]['I']*1e-2, label=Tm_key[0])
plt.xlim([18,40])
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.title('XRD pattern of TmVO4-LS5695 vs pattern calculated from .cif file')
plt.legend()

In [196]:
plt.savefig('./2021-03-12_YTmVO4_analysis/2021-03-12_TmVO4_full_spectrum.pdf')

### Focus on three most intense peaks

In [37]:
# Parameters for zoom on peaks
refs = list(dref.keys())
xlim = {'ax1':[18.7,19.1], 'ax2':[24.5,25.75], 'ax3':[33.25,34.25]}

In [44]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
# fig.subplots_adjust(hspace=0.025)  # adjust space between axes

for sax in ['ax1', 'ax2', 'ax3']:
    ax = eval(sax)
    xref = dref[refs[0]]['2theta']
    Iref = dref[refs[0]]['I']
    ax.plot(xref, Iref/np.max(Iref[np.logical_and(xref>xlim[sax][0], xref<xlim[sax][1])]), 
            label=f'{refs[0]} (calc.)')

    for batch in batches[:1]:
        xxp = dxp[Tm_key]['2theta']
        Ixp = dxp[Tm_key]['I']
        ax.plot(xxp, Ixp/np.max(Ixp[np.logical_and(xxp>xlim[sax][0], xxp<xlim[sax][1])]), 
                label=Tm_key[0])
        
    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim[sax])
    ax.set_ylim([0,1.1])
# ax2.set_xlim(xlim2)
# ax3.set_xlim(xlim3)
ax1.set_ylabel(ylabel)
ax2.set_title('XRD patterns of Y-substituted TmVO4 vs calculated patterns of TmVO4 and YVO4')
plt.legend()

In [197]:
plt.savefig('./2021-03-12_YTmVO4_analysis/2021-03-12_TmVO4_zoom.pdf', bbox_inches='tight')

## Compare measured XRD patterns of Y-substituted TmVO4 vs calculated patterns of TmVO4 and YVO4

In [75]:
measured = [(batch,date) for (batch,date) in dxpk if '2021-03-24' in date]

fig = plt.figure()
plt.plot(dref[refs[0]]['2theta'], dref[refs[0]]['I'], label=f'{refs[0]} (ICSD)')
for key in measured:
    plt.plot(dxp[key]['2theta'], dxp[key]['I']*1e2/max(dxp[key]['I']), label=compounds[key[0]])
plt.plot(dref[refs[1]]['2theta'], dref[refs[1]]['I'], label=f'{refs[1]} (ICSD)')
plt.xlim([15,75])
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.title('XRD patterns of sintered Y-TmVO4 along with reported patterns of TmVO4 and YVO4')
plt.legend()

In [76]:
plt.savefig('./2021-03-30_YTmVO4_analysis/2021-03-30_YTmVO4.pdf', bbox_inches='tight')

In [71]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
# fig.subplots_adjust(hspace=0.025)  # adjust space between axes

xlim = {'ax1':[18.7,19.1], 'ax2':[24.8,25.4], 'ax3':[33.4,34.]}

for sax in ['ax1', 'ax2', 'ax3']:
    ax = eval(sax)
    xref = dref[refs[0]]['2theta']
    Iref = dref[refs[0]]['I']
    ax.plot(xref, Iref/np.max(Iref[np.logical_and(xref>xlim[sax][0], xref<xlim[sax][1])]), 
            label=f'{refs[0]} (ICSD)')

    for key in measured[::-1]:
        xxp = dxp[key]['2theta']
        Ixp = dxp[key]['I']
        ax.plot(xxp, Ixp/np.max(Ixp[np.logical_and(xxp>xlim[sax][0], xxp<xlim[sax][1])]), 
                label=compounds[key[0]])
        
    xref = dref[refs[1]]['2theta']
    Iref = dref[refs[1]]['I']
    ax.plot(xref, Iref/np.max(Iref[np.logical_and(xref>xlim[sax][0], xref<xlim[sax][1])]), 
            label=f'{refs[1]} (ICSD)')

    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim[sax])
    ax.set_ylim([0,1.1])
# ax2.set_xlim(xlim2)
# ax3.set_xlim(xlim3)
ax1.set_ylabel(ylabel)
ax2.set_title('XRD patterns of Y-substituted TmVO4 vs reported patterns of TmVO4 and YVO4')
plt.legend()

In [73]:
plt.savefig('./2021-03-30_YTmVO4_analysis/2021-03-30_YTmVO4_zoom.pdf', bbox_inches='tight')

## Compare XRD patterns before and after sintering

In [84]:
batches = ['LS5696','LS5697']
fig = {}

for idx, batch in enumerate(batches[:]):
    fig[idx], (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)

    xlim = {'ax1':[18.6,19.2], 'ax2':[24.8,25.5], 'ax3':[33.4,34.1]}

    for sax in ['ax1', 'ax2', 'ax3']:
        ax = eval(sax)

        for date in ['2021-03-12','2021-03-24']:
            spectrum = (batch,date)
            xxp = dxp[spectrum]['2theta']
            Ixp = dxp[spectrum]['I']
            ax.plot(xxp, Ixp/np.max(Ixp[np.logical_and(xxp>xlim[sax][0], xxp<xlim[sax][1])]), 
                    label=f'{compounds[batch]},{date}')

        ax.set_xlabel(xlabel)
        ax.set_xlim(xlim[sax])
        ax.set_ylim([0,1.1])

    ax1.set_ylabel(ylabel)
    ax2.set_title('XRD patterns of Y-substituted TmVO4 before and after sintering')
    plt.legend()

In [86]:
plt.savefig('./2021-03-30_YTmVO4_analysis/2021-03-30_15-YTmVO4_sintering.pdf', bbox_inches='tight')
# plt.figure(fig[0].number)
# plt.savefig('./2021-03-30_YTmVO4_analysis/2021-03-30_15-YTmVO4_sintering.pdf', bbox_inches='tight')